# Now Train BC for Air Hockey

In [7]:
from air_hockey_challenge.framework.evaluate_agent import evaluate, custom_evaluate
from baseline.baseline_agent.baseline_agent import build_agent
import pickle

In [29]:
config = {'render': False, 'quiet': True, 'n_episodes': 1000, 'n_cores': 4, 'log_dir': 'logs', 'seed': None, 'generate_score': 'phase-1', 'env_list': ['3dof-hit']}
custom_evaluate(build_agent, **config)

=== CUSTOM EVALUATE ===
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  RE

In [3]:
import pickle

with open("defend_training_data.pkl", "rb") as f:
    training_data = pickle.load(f)
    obs = training_data["obs"]
    actions = training_data["actions"].reshape(training_data["actions"].shape[0],-1)
    next_obs = training_data["next_obs"]
    dones = training_data["dones"]
    info = training_data["info"]

In [4]:
from imitation.data.types import Transitions

transitions = Transitions(obs=obs, acts=actions, infos=info, next_obs=next_obs, dones=dones)

In [5]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper
import gym
import numpy as np

mdp = AirHockeyChallengeWrapper(env="3dof-hit")
obs_space = mdp.info.observation_space
bc_obs_space = gym.spaces.Box(low=obs_space.low, high=obs_space.high, shape=obs_space.shape)


#this deceives u.... action_space from mdp only shows the torque limits
action_space = mdp.info.action_space

jnt_range = mdp.base_env.env_info['robot']['robot_model'].jnt_range
vel_range = mdp.base_env.env_info['robot']['joint_vel_limit'].T * 0.95

low = np.hstack((jnt_range[:,0][:,np.newaxis], vel_range[:,0][:,np.newaxis])).T
high = np.hstack((jnt_range[:,1][:,np.newaxis], vel_range[:,1][:,np.newaxis])).T

bc_action_space = gym.spaces.Box(low=low, high=high, shape=(2,3))

In [33]:
from imitation.algorithms import bc
import numpy as np

rng = np.random.default_rng()
bc_trainer = bc.BC(
    observation_space=bc_obs_space,
    action_space=bc_action_space,
    demonstrations=transitions,
    rng=rng,
    batch_size=256,
    device='cuda'
)

In [35]:
bc_trainer.train(n_epochs=200, progress_bar=True,log_interval=10000,log_rollouts_n_episodes=1000)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 256      |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 88.5     |
|    loss           | 8.12     |
|    neglogp        | 8.13     |
|    prob_true_act  | 0.000318 |
|    samples_so_far | 256      |
--------------------------------


125batch [00:03, 40.87batch/s]
255batch [00:06, 41.56batch/s]
385batch [00:08, 42.63batch/s]
516batch [00:12, 40.54batch/s]
644batch [00:14, 42.58batch/s]
771batch [00:17, 56.18batch/s]
899batch [00:19, 60.45batch/s]
1032batch [00:22, 48.77batch/s][A
1158batch [00:25, 41.39batch/s]
1288batch [00:28, 43.08batch/s]
1417batch [00:31, 42.61batch/s]
1547batch [00:34, 42.57batch/s]
1677batch [00:36, 52.03batch/s]
1806batch [00:39, 59.51batch/s]
1934batch [00:41, 45.04batch/s]
2064batch [00:44, 45.59batch/s]
2189batch [00:47, 40.86batch/s]
2322batch [00:50, 42.72batch/s]
2447batch [00:53, 46.61batch/s]
2578batch [00:55, 41.53batch/s]
2708batch [00:58, 44.00batch/s]
2833batch [01:01, 50.87batch/s]
2964batch [01:04, 42.51batch/s]
3095batch [01:07, 45.81batch/s]
3225batch [01:10, 41.41batch/s]
3351batch [01:13, 44.51batch/s]
3483batch [01:16, 41.92batch/s]
3612batch [01:18, 49.46batch/s]
3738batch [01:21, 43.65batch/s]
3866batch [01:24, 55.73batch/s]
3994batch [01:26, 55.79batch/s]
4127batch [01

--------------------------------
| batch_size        | 256      |
| bc/               |          |
|    batch          | 10000    |
|    ent_loss       | 0.0181   |
|    entropy        | -18.1    |
|    epoch          | 77       |
|    l2_loss        | 0        |
|    l2_norm        | 182      |
|    loss           | -18.2    |
|    neglogp        | -18.2    |
|    prob_true_act  | 4.21e+08 |
|    samples_so_far | 2560256  |
--------------------------------


10061batch [03:43, 45.04batch/s]
10191batch [03:46, 41.39batch/s]
10316batch [03:49, 31.76batch/s]
10446batch [03:52, 48.02batch/s]
10486batch [03:53, 39.94batch/s]

...

12899batch [04:46, 42.40batch/s]
13025batch [04:49, 47.46batch/s]
13154batch [04:51, 58.17batch/s]
13283batch [04:54, 54.26batch/s]
13347batch [04:56, 43.96batch/s]

Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from:  READY  to:  SMASH
Agent:  1 Switch tactic from:  SMASH  to:  READY
Agent:  1 Switch tactic from: 

13412batch [04:57, 40.89batch/s]
13545batch [05:00, 50.26batch/s]
13674batch [05:03, 49.81batch/s]
13800batch [05:05, 45.06batch/s]
13932batch [05:08, 49.29batch/s]
14061batch [05:11, 49.27batch/s]
14186batch [05:13, 43.37batch/s]
14317batch [05:16, 41.37batch/s]
14447batch [05:19, 41.90batch/s]
14576batch [05:22, 45.57batch/s]
14703batch [05:25, 58.88batch/s]
14834batch [05:27, 60.61batch/s]
14960batch [05:30, 47.32batch/s]
15091batch [05:32, 42.30batch/s]
15222batch [05:35, 46.05batch/s]
15348batch [05:38, 51.47batch/s]
15479batch [05:40, 60.02batch/s]
15604batch [05:43, 60.05batch/s]
15734batch [05:45, 49.35batch/s]
15865batch [05:48, 46.20batch/s]
15995batch [05:51, 40.21batch/s]
16124batch [05:53, 44.32batch/s]
16254batch [05:57, 41.25batch/s]
16379batch [06:00, 40.20batch/s]
16508batch [06:03, 42.70batch/s]
16638batch [06:06, 40.36batch/s]
16767batch [06:09, 40.54batch/s]
16899batch [06:12, 44.03batch/s]
17024batch [06:15, 42.15batch/s]
17157batch [06:18, 53.89batch/s]
17286batch

--------------------------------
| batch_size        | 256      |
| bc/               |          |
|    batch          | 20000    |
|    ent_loss       | 0.0189   |
|    entropy        | -18.9    |
|    epoch          | 155      |
|    l2_loss        | 0        |
|    l2_norm        | 192      |
|    loss           | -19.1    |
|    neglogp        | -19.1    |
|    prob_true_act  | 1.3e+09  |
|    samples_so_far | 5120256  |
--------------------------------


20122batch [07:25, 43.18batch/s]
20253batch [07:28, 46.14batch/s]
20378batch [07:31, 41.15batch/s]
20505batch [07:33, 57.23batch/s]
20640batch [07:36, 58.34batch/s]
20764batch [07:38, 52.41batch/s]
20893batch [07:41, 51.79batch/s]
21026batch [07:44, 48.28batch/s]
21154batch [07:46, 40.93batch/s]
21285batch [07:49, 40.92batch/s]
21410batch [07:52, 45.23batch/s]
21540batch [07:55, 43.73batch/s]
21671batch [07:58, 41.09batch/s]
21797batch [08:01, 45.15batch/s]
21930batch [08:04, 45.33batch/s]
22055batch [08:07, 43.21batch/s]
22185batch [08:10, 44.13batch/s]
22315batch [08:13, 40.78batch/s]
22445batch [08:16, 40.63batch/s]
22575batch [08:19, 40.36batch/s]
22700batch [08:22, 41.29batch/s]
22832batch [08:25, 50.77batch/s]
22958batch [08:28, 44.04batch/s]
23088batch [08:31, 45.13batch/s]
23216batch [08:34, 44.44batch/s]
23347batch [08:37, 43.12batch/s]
23477batch [08:40, 40.52batch/s]
23607batch [08:43, 44.57batch/s]
23732batch [08:46, 46.63batch/s]
23863batch [08:49, 45.64batch/s]
23993batch

In [51]:
torch.save(bc_trainer.policy.state_dict(),'intro_il_model.pt')

#### So that didn't end well...

- Now we're going to work with a larger dense neural network first
- Extractors:
    - allow policy and value functions to learn on a more compact & informative representation of state space
    - helps algorithm converge and achieve better performance, especially in high-dim, cont. space
    

In [10]:
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

from gym import spaces
import torch as th
from torch import nn

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy


class AirHockeyNN(nn.Module):
    """
    Custom network for policy and value function.
    It receives as input the features extracted by the features extractor.

    :param feature_dim: dimension of the features extracted with the features_extractor (e.g. features from a CNN)
    :param last_layer_dim_pi: (int) number of units for the last layer of the policy network
    :param last_layer_dim_vf: (int) number of units for the last layer of the value network
    """

    def __init__(
        self,
        feature_dim: int = 12,
        last_layer_dim_pi: int = 32,
        last_layer_dim_vf: int = 32,
    ):
        super().__init__()

        # IMPORTANT:
        # Save output dimensions, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

        # Policy network
        self.policy_net = nn.Sequential(
            nn.Linear(feature_dim, 32), nn.ReLU(),
            nn.Linear(32, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, 64), nn.ReLU(),
            nn.Linear(64, last_layer_dim_pi), nn.ReLU()
        )
        # Value network
        self.value_net = nn.Sequential(
            nn.Linear(feature_dim, 32), nn.ReLU(),
            nn.Linear(32, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, 64), nn.ReLU(),
            nn.Linear(64, last_layer_dim_vf), nn.ReLU()        )

    def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        """
        :return: (th.Tensor, th.Tensor) latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)


class AirHockeyACPolicy(ActorCriticPolicy):
    def __init__(
        self,
        observation_space: spaces.Space,
        action_space: spaces.Space,
        lr_schedule: Callable[[float], float] = lambda _: th.finfo(th.float32).max,
        *args,
        **kwargs,
    ):

        super().__init__(
            observation_space,
            action_space,
            lr_schedule,
            # Pass remaining arguments to base class
            *args,
            **kwargs,
        )
        # Disable orthogonal initialization
        self.ortho_init = False

    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = AirHockeyNN(self.features_dim)

In [12]:
dummy = AirHockeyACPolicy(bc_obs_space, bc_action_space)

In [14]:
from imitation.algorithms import bc
import numpy as np

rng = np.random.default_rng()
bc_trainer = bc.BC(
    observation_space=bc_obs_space,
    action_space=bc_action_space,
    demonstrations=transitions,
    rng=rng,
    batch_size=256,
    policy=dummy,
    device='cpu'
)

In [15]:
bc_trainer.train(n_epochs=200, progress_bar=True,log_interval=10000,log_rollouts_n_episodes=1000)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 256      |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 665      |
|    loss           | 8.02     |
|    neglogp        | 8.03     |
|    prob_true_act  | 0.000332 |
|    samples_so_far | 256      |
--------------------------------


1356batch [00:09, 146.92batch/s]
2712batch [00:19, 144.71batch/s]
4067batch [00:28, 136.13batch/s]
5435batch [00:38, 143.91batch/s]
6792batch [00:48, 144.27batch/s]
8148batch [00:58, 144.59batch/s]
9519batch [01:07, 144.71batch/s]
9996batch [01:11, 142.30batch/s]

--------------------------------
| batch_size        | 256      |
| bc/               |          |
|    batch          | 10000    |
|    ent_loss       | 0.017    |
|    entropy        | -17      |
|    epoch          | 7        |
|    l2_loss        | 0        |
|    l2_norm        | 847      |
|    loss           | -17.5    |
|    neglogp        | -17.6    |
|    prob_true_act  | 1.83e+08 |
|    samples_so_far | 2560256  |
--------------------------------


10878batch [01:17, 142.02batch/s]
12163batch [01:27, 139.55batch/s]


KeyboardInterrupt: 

# Test Behavior Cloning in Air Hockey Challenge Core

In [1]:
from util_networks import AirHockeyACPolicy
import torch
import numpy as np

In [2]:
from air_hockey_challenge.framework import AgentBase
from air_hockey_challenge.framework.challenge_core import ChallengeCore, CustomChallengeCore

class BehaviorCloneAgent(AgentBase):
    def __init__(self, env_info, model, device, **kwargs):
        super().__init__(env_info, **kwargs)
        self.new_start = True
        self.hold_position = None
        
        self.model = model
        self.device = device

    def reset(self):
        self.new_start = True
        self.hold_position = None

    def draw_action(self, observation):
        if self.new_start:
            self.new_start = False
            self.hold_position = self.get_joint_pos(observation)

            velocity = np.zeros_like(self.hold_position)
            action = np.vstack([self.hold_position, velocity])
        else:
            obs_tensor = torch.tensor(observation,dtype=torch.float).reshape(1,12).detach()
            action,_,_ = self.model(obs_tensor)
            action = action.reshape(2,3).detach().numpy()
            
        
        return action

In [3]:
from imitation.algorithms.bc import reconstruct_policy

model = reconstruct_policy("defense_bc_policy2_1024").cpu()

In [4]:
model

AirHockeyACPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): AirHockeyNN(
    (policy_net): Sequential(
      (0): Linear(in_features=12, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=128, bias=True)
      (5): ReLU()
      (6): Linear(in_features=128, out_features=256, bias=True)
      (7): ReLU()
      (8): Linear(in_features=256, out_features=64, bias=True)
      (9): ReLU()
      (10): Linear(in_features=64, out_features=32, bias=True)
      (11): ReLU()
    )
    (value_net): Sequential(
      (0): Linear(in_features=12, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=

In [5]:
action, _, _ = model.forward(torch.zeros(1,12))
action

tensor([[[-0.7112,  1.0207,  1.0207],
         [ 0.9087,  1.2793, -0.4331]]], grad_fn=<ReshapeAliasBackward0>)

In [6]:
def build_agent(env_info, **kwargs):
    """
    Function where an Agent that controls the environments should be returned.
    The Agent should inherit from the mushroom_rl Agent base env.

    Args:
        env_info (dict): The environment information
        kwargs (any): Additionally setting from agent_config.yml
    Returns:
         (AgentBase) An instance of the Agent
    """

    return BehaviorCloneAgent(env_info, **kwargs)

In [7]:
from air_hockey_challenge.framework.air_hockey_challenge_wrapper import AirHockeyChallengeWrapper

mdp = AirHockeyChallengeWrapper(env="3dof-defend", action_type="position-velocity", interpolation_order=3, debug=True)
agent = BehaviorCloneAgent(mdp.base_env.env_info, model, 'cpu')

In [8]:
from air_hockey_challenge.framework.evaluate_agent import evaluate
evaluate(build_agent, 'some_folder/', ['3dof-defend'], 100, 1, model=model, device='cpu', render=True)

Environment:        3dof-defend
Number of Episodes: 100
Success:            0.0300
Penalty:            525
Number of Violations: 
  joint_vel_constr  100
  ee_constr         79
  Jerk              100
  joint_pos_constr  44
  Total             323
-------------------------------------------------

